In [32]:
import requests
import json
import time
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [33]:
#API key
api_key = "RGAPI-ad2f295e-47e7-4fce-9289-4e1bc1ab65bc"
gameName = "Slylio" #username
tagLine = "EUW" #suffixe

In [47]:
#Player unviveraly unique identifier
requestPUUID = requests.get(f"https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}")

#Exctract puuid from the request
puuid = requestPUUID.json()["puuid"] 


{'puuid': 't7tBZiapGX53Mzo-UKqbkZiEHjuCcODEL8HiyRZroop4TFvv69E1FSIV2oYAldZdpsFMQrA0gHJCfA', 'gameName': 'Slylio', 'tagLine': 'EUW'}


In [48]:
#Match history from PUUID 
nb_of_games = 20
games = requests.get(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count={nb_of_games}&api_key={api_key}")
print(games.json())

['EUW1_6820269222', 'EUW1_6818243881', 'EUW1_6817876234', 'EUW1_6817832257', 'EUW1_6817791170', 'EUW1_6817751054', 'EUW1_6817736140', 'EUW1_6817704800', 'EUW1_6817685454', 'EUW1_6817654890', 'EUW1_6815607325', 'EUW1_6815553132', 'EUW1_6815479868', 'EUW1_6815289763', 'EUW1_6812960275', 'EUW1_6812875294', 'EUW1_6812354586', 'EUW1_6812293142', 'EUW1_6811843539', 'EUW1_6811458138']


In [56]:
#Extract matchs data from match ids
matchs = games.json()
data = pd.DataFrame()
cout=0
for match in matchs:
    cout+=1
    print("Match n°",cout,"/",nb_of_games,"\r",end="")
    match_data = requests.get(f"https://europe.api.riotgames.com/lol/match/v5/matches/{match}?api_key={api_key}")
    
print(data)

[{'metadata': {'dataVersion': '2', 'matchId': 'EUW1_6820269222', 'participants': ['uAFEuYDjPl9t2rQiv27E2Vq7HJtHsJ-393jODOKykVAmzoYNa5vvMplEsvnTkSfal5N7-sWtYHcc9Q', 't7tBZiapGX53Mzo-UKqbkZiEHjuCcODEL8HiyRZroop4TFvv69E1FSIV2oYAldZdpsFMQrA0gHJCfA', '2oQaI_vD2SMOqI6iSX-r527AYYm4D2OjBd6TbWJ9AIPKdIc9zSiPGjUhU_S9Nr2mTTPmdtYAwCAN8Q', 'sQGR8a5KEZu31Q-qyLax6R7SESaIsxPCfm0tkjn6qX7mKwRuGqiCWMYZNMDq8-KWUKOls_lxmOEPoA', 'EaelV6vN9scnKG5BZ0RADgqeJeJp2wC6ZIVMKLhi7KQI_rPnBV0PWPOLDGsx46H3n-v8Nw-IsQnJHQ', 'r4cR5i-TiXhe9dKk4oqiOdGvmy6yf7tscCuLA6GxRRYvHxb8otrIIkj5kxNdEA6HG5uYw1eyc1kbWQ', 'O9gFOuA0QC-eiBIeBJO7a4L1-xOwQIvH1v1WhA2rylO55hab88DMMDH3gVtdVFc0SAza3FZbQuz37g', 'yxLGYZg3_ejIw48W1gNnCkyv2EwAkOTKaHb-szMXUz0SWyTDS9jZYyfnX1VecICF2N5hax-8CqfFNA', 'DtvBEf_YW9PmSQAiwH7JfgkfC_Ya27P0eaqOKPhEm1t3E0ZCTDkEPByifgpmXMcA3_BUYAdYicP6Rg', 'CgYCF3n9lxyuJRcQ6KsjG5PP3UXjGM61kOK6i82PR4NSNP_9iPKCVvJ-f4VjBvgT_Shv-O_pgER5Qw']}, 'info': {'endOfGameResult': 'GameComplete', 'gameCreation': 1708206035723, 'gameDuration': 2195,

                                             metadata  \
0   {'dataVersion': '2', 'matchId': 'EUW1_68202692...   
1   {'dataVersion': '2', 'matchId': 'EUW1_68182438...   
2   {'dataVersion': '2', 'matchId': 'EUW1_68178762...   
3   {'dataVersion': '2', 'matchId': 'EUW1_68178322...   
4   {'dataVersion': '2', 'matchId': 'EUW1_68177911...   
5   {'dataVersion': '2', 'matchId': 'EUW1_68177510...   
6   {'dataVersion': '2', 'matchId': 'EUW1_68177361...   
7   {'dataVersion': '2', 'matchId': 'EUW1_68177048...   
8   {'dataVersion': '2', 'matchId': 'EUW1_68176854...   
9   {'dataVersion': '2', 'matchId': 'EUW1_68176548...   
10  {'dataVersion': '2', 'matchId': 'EUW1_68156073...   
11  {'dataVersion': '2', 'matchId': 'EUW1_68155531...   
12  {'dataVersion': '2', 'matchId': 'EUW1_68154798...   
13  {'dataVersion': '2', 'matchId': 'EUW1_68152897...   
14  {'dataVersion': '2', 'matchId': 'EUW1_68129602...   
15  {'dataVersion': '2', 'matchId': 'EUW1_68128752...   
16  {'dataVersion': '2', 'match

In [ ]:
#DEMI CLASSIFIER